<a href="https://colab.research.google.com/github/maleehamcgill/Customer-Reviews-Insights-Pipeline/blob/app/main_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from fastapi import FastAPI, Depends
from sqlalchemy.orm import Session
from .database import engine, Base, get_db
from .models import Review, ReviewAnalysis
from sqlalchemy import func

Base.metadata.create_all(bind=engine)

app = FastAPI(title="Customer Review Insights API")

@app.get("/")
def root():
    return {"message": "API running"}

@app.get("/latest")
def get_latest_reviews(limit: int = 10, db: Session = Depends(get_db)):
    reviews = db.query(Review).order_by(Review.review_date.desc()).limit(limit).all()
    result = []
    for review in reviews:
        result.append({
            "review_text": review.review_text,
            "sentiment": review.analysis[0].sentiment if review.analysis else None,
            "topics": review.analysis[0].topics if review.analysis else None
        })
    return result

@app.get("/sentiment")
def get_sentiment_summary(db: Session = Depends(get_db)):
    result = db.query(ReviewAnalysis.sentiment, func.count(ReviewAnalysis.id))\
        .group_by(ReviewAnalysis.sentiment).all()
    return {sent: count for sent, count in result}

@app.get("/topics")
def get_topics_summary(db: Session = Depends(get_db)):
    topics = db.query(ReviewAnalysis.topics).all()
    topic_count = {}
    for row in topics:
        if row.topics:
            for topic in row.topics.split(","):
                topic = topic.strip().lower()
                topic_count[topic] = topic_count.get(topic, 0) + 1
    return topic_count
